# Havi rendszeres lekérdezések futtatása

In [1]:
import pyodbc
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime, timezone
import matplotlib.pyplot as plt

%matplotlib inline

## Havi változás-halmaz adatcsomag előállítása -> .csv

In [2]:
df_already_scanned = pd.read_csv("already_scanned.csv", sep=";")

In [3]:
paramList = []
entry = []
x = len(df_already_scanned)

In [4]:
entry.append('sumRecord_Number')
entry.append(df_already_scanned['sourceRecordNumber'].sum())
paramList.append(entry)
entry = []
entry.append('minSumPriceBill')
entry.append(df_already_scanned['dfSumPriceBill'].min())
paramList.append(entry)
entry = []
entry.append('maxSumPriceBill')
entry.append(df_already_scanned['dfSumPriceBill'].max())
paramList.append(entry)
entry = []
entry.append('avgSumPriceBill')
entry.append(df_already_scanned['dfSumPriceBill'].mean())
paramList.append(entry)
entry = []

In [5]:
penultSumPriceBill = df_already_scanned.loc[x-2:x-2, ['dfSumPriceBill']].iloc[0,0]
entry.append('penultSumPriceBill')
entry.append(penultSumPriceBill)
paramList.append(entry)
entry = []

lastSumPriceBill = df_already_scanned.loc[x-1:x, ['dfSumPriceBill']].iloc[0,0]
entry.append('lastSumPriceBill')
entry.append(lastSumPriceBill)
paramList.append(entry)
entry = []

changeLastSumPriceBill = ((lastSumPriceBill/penultSumPriceBill)-1)*100
entry.append('changeLastSumPriceBill')
entry.append(changeLastSumPriceBill)
paramList.append(entry)
entry = []

penultSumBoxMill = df_already_scanned.loc[x-2:x-2, ['dfSumBoxMill']].iloc[0,0]
entry.append('penultSumBoxMill')
entry.append(penultSumBoxMill)
paramList.append(entry)
entry = []

lastSumBoxMill = df_already_scanned.loc[x-1:x, ['dfSumBoxMill']].iloc[0,0]
entry.append('lastSumBoxMill')
entry.append(lastSumBoxMill)
paramList.append(entry)
entry = []

changeLastSumBoxMill = ((lastSumBoxMill/penultSumBoxMill)-1)*100
entry.append('changeLastSumBoxMill')
entry.append(changeLastSumBoxMill)
paramList.append(entry)
entry = []

yesterSumBoxMill = df_already_scanned.loc[x-13:x-12, ['dfSumBoxMill']].iloc[0,0]
entry.append('yesterSumBoxMill')
entry.append(yesterSumBoxMill)
paramList.append(entry)
entry = []

changeYesterSumBoxMill = ((lastSumBoxMill/yesterSumBoxMill)-1)*100
entry.append('changeYesterSumBoxMill')
entry.append(changeYesterSumBoxMill)
paramList.append(entry)
entry = []


yesterSumPriceBill = df_already_scanned.loc[x-13:x-12, ['dfSumPriceBill']].iloc[0,0]
entry.append('yesterSumPriceBill')
entry.append(yesterSumPriceBill)
paramList.append(entry)
entry = []


changeYesterSumPriceBill = ((lastSumPriceBill/yesterSumPriceBill)-1)*100
entry.append('changeYesterSumPriceBill')
entry.append(changeYesterSumPriceBill)
paramList.append(entry)
entry = []

In [6]:
neak_paramList = pd.DataFrame(paramList, columns =['parameter', 'value'])

In [7]:
neak_paramList.to_csv("neak_paramList.csv", sep=";", encoding='utf-8', index=False)

## Havi felösszegzések előállítása recent7 -> df_recent7_monthly

In [2]:
# Abbreviations of the Names of the Months (Yale University Library)
# https://web.library.yale.edu/cataloging/months
# line = "English"

monthDict = {
'01':"Jan.",
'02':"Feb.",
'03':"Mar.",
'04':"Apr.",
'05':"May",
'06':"June",
'07':"July",
'08':"Aug.",
'09':"Sept.",
'10':"Oct.",
'11':"Nov.",
'12':"Dec."}

In [4]:
db = sqlite3.connect('neak2.sqlite3')

In [5]:
q = "select idoszak, sum(fogyar) as sumfogyar, sum(doboz) as sumdoboz from recent7 group by idoszak;"

In [6]:
df = pd.read_sql(q, db)

In [8]:
df['year']= df['idoszak'].str.slice(0, 4)

In [9]:
df['honap']= df['idoszak'].str.slice(4, 6)

In [10]:
df['month'] = df['honap'].map(monthDict)

In [23]:
# df.to_csv("recent6_monthly.csv", sep=";", encoding='utf-8', index=False)

In [13]:
df.to_sql(("df_recent7_monthly"), db, if_exists = 'replace')

In [14]:
db.close()

## Havi felösszegzések előállítása whole2022 -> df_atc1_recent

In [27]:
db = sqlite3.connect('neak2.sqlite3')

In [28]:
q = "SELECT idoszak, substr(atc, 1, 1) AS atc1, sum(doboz) AS sumdob, sum(fogyar) AS sumfogyar FROM whole2022 GROUP BY substr(atc, 1, 1), idoszak;"

In [29]:
df = pd.read_sql(q, db)

In [30]:
df

,idoszak,atc1,sumdob,sumfogyar
0,202202,-,5.150000,9.021890e+06
1,202204,-,1.000000,1.804349e+06
2,202205,-,5.000000,3.608698e+06
3,202206,-,8.750000,3.615644e+06
4,202207,-,3.000000,3.609538e+06
...,...,...,...,...
121,202204,V,409392.064689,1.631803e+09
122,202205,V,416376.177128,1.674671e+09
123,202206,V,421033.719723,1.702956e+09
124,202207,V,404909.643424,1.649154e+09


In [33]:
df_atc1 = df.pivot(index='atc1', columns='idoszak', values='sumdob').fillna(0)

In [34]:
df_atc1ft = df.pivot(index='atc1', columns='idoszak', values='sumfogyar').fillna(0)

In [35]:
df_atc1

idoszak,202201,202202,202203,202204,202205,202206,202207,202208
atc1,,,,,,,,
-,0.000000e+00,5.150000e+00,0.000000e+00,1.000000e+00,5.000000e+00,8.750000e+00,3.000000e+00,1.000000e+00
A,2.822996e+06,2.854347e+06,3.076204e+06,2.740975e+06,2.831275e+06,2.825518e+06,2.673257e+06,2.836903e+06
B,1.027582e+06,1.013524e+06,1.115631e+06,1.012483e+06,1.069163e+06,1.075053e+06,1.013221e+06,1.095575e+06
C,5.590668e+06,5.540089e+06,6.179563e+06,5.573265e+06,5.912467e+06,5.938523e+06,5.542669e+06,5.846233e+06
D,2.458376e+05,2.452091e+05,2.905054e+05,2.495954e+05,2.691559e+05,2.581992e+05,2.354804e+05,2.360074e+05
G,2.318122e+05,2.285869e+05,2.576791e+05,2.325525e+05,2.472390e+05,2.474284e+05,2.245441e+05,2.377548e+05
H,2.008414e+05,1.989373e+05,2.283020e+05,2.016805e+05,2.118293e+05,2.090617e+05,1.913742e+05,1.992098e+05
I,1.436083e+06,1.305168e+06,1.512923e+06,1.339834e+06,1.388955e+06,1.261753e+06,1.121245e+06,1.187223e+06
J,6.574399e+05,6.568671e+05,7.590046e+05,5.926986e+05,5.430780e+05,4.735648e+05,4.586289e+05,4.821281e+05


In [36]:
df_atc1ft

idoszak,202201,202202,202203,202204,202205,202206,202207,202208
atc1,,,,,,,,
-,0.000000e+00,9.021890e+06,0.000000e+00,1.804349e+06,3.608698e+06,3.615644e+06,3.609538e+06,1.804349e+06
A,9.070201e+09,9.237626e+09,1.023046e+10,9.223284e+09,9.729720e+09,9.905000e+09,9.465499e+09,9.995631e+09
B,5.585106e+09,5.486753e+09,6.142151e+09,5.672033e+09,6.103235e+09,6.147394e+09,5.815425e+09,6.322889e+09
C,7.941015e+09,7.884912e+09,8.819301e+09,8.013763e+09,8.518707e+09,8.542958e+09,7.989127e+09,8.513935e+09
D,4.504471e+08,4.491802e+08,5.179559e+08,4.741243e+08,5.162438e+08,5.143592e+08,4.658089e+08,4.838408e+08
G,1.118440e+09,1.099868e+09,1.218036e+09,1.136669e+09,1.224166e+09,1.189387e+09,9.583523e+08,1.066668e+09
H,9.049444e+08,8.948981e+08,9.979394e+08,9.105553e+08,9.470900e+08,9.315195e+08,8.715443e+08,8.969208e+08
I,7.217619e+08,6.438635e+08,7.438103e+08,7.239512e+08,7.517459e+08,6.960119e+08,6.450958e+08,6.600117e+08
J,2.013880e+09,1.953068e+09,2.296347e+09,1.873182e+09,1.844895e+09,1.729218e+09,1.782991e+09,2.290598e+09


In [31]:
# df.to_csv("neak_atc1.csv", sep=";", encoding='utf-8', index=False)

In [37]:
df_atc1.to_sql(("df_atc1_2022_dob"), db, if_exists = 'replace')

In [38]:
df_atc1ft.to_sql(("df_atc1_2022_ft"), db, if_exists = 'replace')

In [39]:
db.close()

## Havi felösszegzések előállítása whole2022 -> df_atc_idoszak

In [53]:
db = sqlite3.connect('neak2.sqlite3')

In [54]:
q = "SELECT idoszak, atc, sum(doboz) AS sumdob, sum(fogyar) AS sumfogyar FROM whole2022 GROUP BY idoszak, atc;"

In [60]:
df = pd.read_sql(q, db)

In [61]:
df['atc1']= df['atc'].str.slice(0, 1)

In [62]:
df['atc3']= df['atc'].str.slice(0, 3)

In [63]:
df['atc5']= df['atc'].str.slice(0, 5)

In [64]:
df

,idoszak,atc,sumdob,sumfogyar,atc1,atc3,atc5
0,202201,A02AD01,0.400000,7.176000e+02,A,A02,A02AD
1,202201,A02BA03,241514.024142,2.824190e+08,A,A02,A02BA
2,202201,A02BC01,30116.593008,6.224947e+07,A,A02,A02BC
3,202201,A02BC02,605232.907428,4.644190e+08,A,A02,A02BC
4,202201,A02BC03,30049.500000,2.963353e+07,A,A02,A02BC
...,...,...,...,...,...,...,...
6939,202208,V06C,167651.000000,4.518593e+08,V,V06,V06C
6940,202208,V06D,248657.736200,1.195516e+09,V,V06,V06D
6941,202208,V06DD,1846.000000,6.777034e+07,V,V06,V06DD
6942,202208,V07AB,145.920000,1.105660e+04,V,V07,V07AB


In [65]:
df.to_sql(("df_atc_idoszak"), db, if_exists = 'replace')

In [66]:
db.close()

## Havi felösszegzések előállítása whole2022 -> df_megye22

In [67]:
db = sqlite3.connect('neak2.sqlite3')

In [68]:
q = "SELECT idoszak, megye, sum(doboz) AS sumdob, sum(fogyar) AS sumfogyar FROM whole2022 GROUP BY idoszak, megye;"

In [69]:
df = pd.read_sql(q, db)

In [70]:
# https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwjki_OS-5v5AhVN2aQKHajIDVkQFnoECBEQAQ&url=https%3A%2F%2Fwww.ksh.hu%2Fdocs%2Fhun%2Fxftp%2Froviditesek.pdf&usg=AOvVaw2_-SdIQNO8KCskUhnIm9gd
# KSH módszertani útmutató
# Megyék rövidítésjegyzéke

megyedict = {"BA":"Baranya",
"BO":"Borsod-Abaúj-Zemplén",
"BP":"Budapest",
"BÁ":"Bács-Kiskun",
"BÉ":"Békés",
"CS":"Csongrád-Csanád",
"FE":"Fejér",
"GY":"Győr-Moson-Sopron",
"HA":"Hajdú-Bihar",
"HE":"Heves",
"JÁ":"Jász-Nagykun-Szolnok",
"KO":"Komárom-Esztergom",
"NÓ":"Nógrád",
"PE":"Pest",
"SO":"Somogy",
"SZ":"Szabolcs-Szatmár-Bereg",
"TO":"Tolna",
"VA":"Vas",
"VE":"Veszprém",
"ZA":"Zala"}

In [71]:
df['megyekod'] = df['megye']

In [72]:
df['megye'] = df['megye'].map(megyedict)

In [73]:
df.sort_values(by=['megye','idoszak'], inplace=True)

In [75]:
df_megye = df.pivot(index='megye', columns='idoszak', values='sumfogyar').fillna(0)

In [76]:
df_megye

idoszak,202201,202202,202203,202204,202205,202206,202207,202208
megye,,,,,,,,
Baranya,2.069489e+09,2.029810e+09,2.253558e+09,2.095831e+09,2.122478e+09,2.168494e+09,2.087155e+09,2.252080e+09
Borsod-Abaúj-Zemplén,2.710811e+09,2.706992e+09,3.023537e+09,2.803326e+09,2.898583e+09,2.852389e+09,2.745008e+09,2.937986e+09
Budapest,1.020327e+10,1.012598e+10,1.152048e+10,1.037299e+10,1.105983e+10,1.106277e+10,1.008731e+10,1.131799e+10
Bács-Kiskun,2.360597e+09,2.329062e+09,2.594694e+09,2.378819e+09,2.478937e+09,2.464113e+09,2.340958e+09,2.509216e+09
Békés,1.609036e+09,1.571747e+09,1.763352e+09,1.602624e+09,1.699450e+09,1.688135e+09,1.569271e+09,1.685821e+09
Csongrád-Csanád,2.371913e+09,2.280868e+09,2.566633e+09,2.319685e+09,2.493944e+09,2.531321e+09,2.376540e+09,2.490419e+09
Fejér,1.779194e+09,1.748205e+09,1.957052e+09,1.784828e+09,1.911927e+09,1.904843e+09,1.773221e+09,1.887893e+09
Győr-Moson-Sopron,2.134032e+09,2.071761e+09,2.349700e+09,2.110893e+09,2.263826e+09,2.225364e+09,2.107063e+09,2.282353e+09
Hajdú-Bihar,2.633794e+09,2.581376e+09,2.883148e+09,2.616127e+09,2.736178e+09,2.701251e+09,2.595215e+09,2.770404e+09


In [77]:
#df.to_csv("neak_megye.csv", sep=";", encoding='utf-8', index=False)
df_megye.to_sql(("df_megye22"), db, if_exists = 'replace')

In [78]:
db.close()